In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os

In [2]:
column_names = ['unit_number', 'time_cycle', 
                'setting_1', 'setting_2', 'setting_3',
                'sensor_1', 'sensor_2', 'sensor_3', 'sensor_4', 'sensor_5',
                'sensor_6', 'sensor_7', 'sensor_8', 'sensor_9', 'sensor_10',
                'sensor_11', 'sensor_12', 'sensor_13', 'sensor_14', 'sensor_15',
                'sensor_16', 'sensor_17', 'sensor_18', 'sensor_19', 'sensor_20',
                'sensor_21']

# Define paths
train_path = '/workspaces/Engine_Predictive_System/data/train_FD001.txt'

# Load the training data
df_train = pd.read_csv(train_path, 
                       sep='\s+', 
                       header=None, 
                       names=column_names)

df_train = df_train.dropna(axis='columns', how='all')

print(f"Loaded training data with {df_train.shape[0]} rows.")
print(df_train.head())

Loaded training data with 20631 rows.
   unit_number  time_cycle  setting_1  setting_2  setting_3  sensor_1  \
0            1           1    -0.0007    -0.0004      100.0    518.67   
1            1           2     0.0019    -0.0003      100.0    518.67   
2            1           3    -0.0043     0.0003      100.0    518.67   
3            1           4     0.0007     0.0000      100.0    518.67   
4            1           5    -0.0019    -0.0002      100.0    518.67   

   sensor_2  sensor_3  sensor_4  sensor_5  ...  sensor_12  sensor_13  \
0    641.82   1589.70   1400.60     14.62  ...     521.66    2388.02   
1    642.15   1591.82   1403.14     14.62  ...     522.28    2388.07   
2    642.35   1587.99   1404.20     14.62  ...     522.42    2388.03   
3    642.35   1582.79   1401.87     14.62  ...     522.86    2388.08   
4    642.37   1582.85   1406.22     14.62  ...     522.19    2388.04   

   sensor_14  sensor_15  sensor_16  sensor_17  sensor_18  sensor_19  \
0    8138.62     8.

<>:14: SyntaxWarning: invalid escape sequence '\s'
<>:14: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_17742/3206098320.py:14: SyntaxWarning: invalid escape sequence '\s'
  sep='\s+',


In [ ]:

# Let's check for any sensors that have zero variance (i.e., they never change)
# These sensors provide no information and should be dropped.

sensor_variance = df_train.var()
print("--- Sensor Variance ---")
print(sensor_variance)

# Identify columns with zero variance (or very close to it)
# These are often 'setting_3', 'sensor_1', 'sensor_5', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19'
# Let's find them automatically
cols_to_drop = sensor_variance[sensor_variance < 1e-10].index.tolist()

# Also, 'setting_3' is often constant and can be dropped if it's in there
if 'setting_3' not in cols_to_drop:
     cols_to_drop.append('setting_3')

# We also drop the 'settings' as they aren't sensors
cols_to_drop.extend(['setting_1', 'setting_2'])
        
# Let's make sure we don't accidentally drop essential columns
cols_to_drop = [col for col in cols_to_drop if col in df_train.columns and col not in ['unit_number', 'time_cycle']]
        
# Remove duplicates
cols_to_drop = list(set(cols_to_drop)) 

print(f"\nDropping {len(cols_to_drop)} useless/constant columns:")
print(cols_to_drop)

# Drop the columns
df_train_cleaned = df_train.drop(columns=cols_to_drop)

print("\n--- Cleaned Data (First 5 Rows) ---")
print(df_train_cleaned.head())

--- Sensor Variance ---
unit_number    8.542545e+02
time_cycle     4.744591e+03
setting_1      4.784340e-06
setting_2      8.588541e-08
setting_3      0.000000e+00
sensor_1       0.000000e+00
sensor_2       2.500533e-01
sensor_3       3.759099e+01
sensor_4       8.101089e+01
sensor_5       2.840037e-29
sensor_6       1.929279e-06
sensor_7       7.833883e-01
sensor_8       5.038938e-03
sensor_9       4.876536e+02
sensor_10      0.000000e+00
sensor_11      7.133568e-02
sensor_12      5.439850e-01
sensor_13      5.172330e-03
sensor_14      3.639005e+02
sensor_15      1.406628e-03
sensor_16      1.203765e-35
sensor_17      2.398667e+00
sensor_18      0.000000e+00
sensor_19      0.000000e+00
sensor_20      3.266927e-02
sensor_21      1.171825e-02
dtype: float64

Dropping 9 useless/constant columns:
['sensor_18', 'setting_3', 'sensor_1', 'sensor_5', 'setting_2', 'sensor_10', 'sensor_16', 'setting_1', 'sensor_19']

--- Cleaned Data (First 5 Rows) ---
   unit_number  time_cycle  sensor_2  sens

In [ ]:

# Our data shows an engine's life *up to failure*.
# We need to calculate the RUL for every single row.
# RUL = (Total cycles for that engine) - (Current cycle for that row)

# 1. Find the maximum (failure) cycle for each engine
df_max_cycles = df_train_cleaned.groupby('unit_number')['time_cycle'].max().reset_index()
df_max_cycles.columns = ['unit_number', 'max_cycle']

# 2. Merge this max_cycle value back onto the main dataframe
df_train_rul = df_train_cleaned.merge(df_max_cycles, on='unit_number', how='left')

# 3. Create the 'RUL' column
df_train_rul['RUL'] = df_train_rul['max_cycle'] - df_train_rul['time_cycle']

# 4. Drop the temporary 'max_cycle' column
df_train_rul = df_train_rul.drop(columns=['max_cycle'])

print("\n--- Data with RUL Engineered (First 5 Rows) ---")
print(df_train_rul.head())

# Spot check: Look at the last few rows for engine 1. The RUL should count down to 0.
print("\n--- Spot Check: Engine 1 (End of Life) ---")
print(df_train_rul[df_train_rul['unit_number'] == 1].tail())


--- Data with RUL Engineered (First 5 Rows) ---
   unit_number  time_cycle  sensor_2  sensor_3  sensor_4  sensor_6  sensor_7  \
0            1           1    641.82   1589.70   1400.60     21.61    554.36   
1            1           2    642.15   1591.82   1403.14     21.61    553.75   
2            1           3    642.35   1587.99   1404.20     21.61    554.26   
3            1           4    642.35   1582.79   1401.87     21.61    554.45   
4            1           5    642.37   1582.85   1406.22     21.61    554.00   

   sensor_8  sensor_9  sensor_11  sensor_12  sensor_13  sensor_14  sensor_15  \
0   2388.06   9046.19      47.47     521.66    2388.02    8138.62     8.4195   
1   2388.04   9044.07      47.49     522.28    2388.07    8131.49     8.4318   
2   2388.08   9052.94      47.27     522.42    2388.03    8133.23     8.4178   
3   2388.11   9049.48      47.13     522.86    2388.08    8133.83     8.3682   
4   2388.06   9055.15      47.28     522.19    2388.04    8133.80     

In [ ]:

# Now we save this fully-prepared DataFrame into a clean SQL database.
# This database will be the "single source of truth" for the rest of our project.

# Define the database path. We'll store it in the 'data' folder.
db_path = '/workspaces/Engine_Predictive_System/data/turbofan.db'
table_name = 'train_data_FD001'

# Create a connection
conn = sqlite3.connect(db_path)
print(f"\nConnecting to SQL database at {db_path}...")

# Use pandas .to_sql() to write the DataFrame to the SQL table
# if_exists='replace' means it will overwrite the table if it already exists.
df_train_rul.to_sql(table_name, conn, if_exists='replace', index=False)

print(f"Successfully wrote {df_train_rul.shape[0]} rows to table '{table_name}'.")

# Close the connection
conn.close()


Connecting to SQL database at /workspaces/Engine_Predictive_System/data/turbofan.db...
Successfully wrote 20631 rows to table 'train_data_FD001'.


In [ ]:

# Let's read the data *back* from the SQL DB to prove it worked.

print("\n--- Verifying SQL Database ---")
try:
    conn = sqlite3.connect(db_path)
    
    # Run a test query
    query = f"SELECT * FROM {table_name} WHERE unit_number = 1 AND time_cycle < 5"
    df_from_db = pd.read_sql_query(query, conn)
    
    conn.close()
    
    print("Test query successful! Data from DB:")
    print(df_from_db)

except Exception as e:
    print(f"An error occurred during verification: {e}")


--- Verifying SQL Database ---
Test query successful! Data from DB:
   unit_number  time_cycle  sensor_2  sensor_3  sensor_4  sensor_6  sensor_7  \
0            1           1    641.82   1589.70   1400.60     21.61    554.36   
1            1           2    642.15   1591.82   1403.14     21.61    553.75   
2            1           3    642.35   1587.99   1404.20     21.61    554.26   
3            1           4    642.35   1582.79   1401.87     21.61    554.45   

   sensor_8  sensor_9  sensor_11  sensor_12  sensor_13  sensor_14  sensor_15  \
0   2388.06   9046.19      47.47     521.66    2388.02    8138.62     8.4195   
1   2388.04   9044.07      47.49     522.28    2388.07    8131.49     8.4318   
2   2388.08   9052.94      47.27     522.42    2388.03    8133.23     8.4178   
3   2388.11   9049.48      47.13     522.86    2388.08    8133.83     8.3682   

   sensor_17  sensor_20  sensor_21  RUL  
0        392      39.06    23.4190  191  
1        392      39.00    23.4236  190  
2  